In [ ]:
!pip install matplotlib


In [1]:
import tensorflow as tf
import os

# Path to your dataset folder (each subfolder corresponds to a different person)
data_dir = "C:\\Users\\PCD\\Desktop\\rtrain"  # Replace with your dataset path

# Parameters
batch_size = 32
img_height = 64
img_width = 64
epochs = 20             # Increase epochs if needed
validation_split = 0.2  # 20% of data will be used for validation
seed = 123              # For reproducibility

# -------------------------------
# 1) Load the training dataset
# -------------------------------
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=validation_split,
    subset="training",
    seed=seed,
    image_size=(img_height, img_width),
    batch_size=batch_size,
)

# Retrieve and store class names BEFORE caching/prefetching
class_names = train_ds.class_names
num_classes = len(class_names)
print("Detected classes:", num_classes)
print("Class names:", class_names)

# Optionally save class names to a text file
with open("class_names.txt", "w") as f:
    for name in class_names:
        f.write(name + "\n")

# -------------------------------
# 2) Load the validation dataset
# -------------------------------
val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    data_dir,
    validation_split=validation_split,
    subset="validation",
    seed=seed,
    image_size=(img_height, img_width),
    batch_size=batch_size,
)

# -------------------------------
# 3) Optimize dataset performance
# -------------------------------
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

# -------------------------------
# 4) Data Augmentation
# -------------------------------
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
])

# -------------------------------
# 5) Build the CNN model
# -------------------------------
model = tf.keras.models.Sequential([
    # Apply data augmentation
    data_augmentation,
    
    # Rescale pixel values to [0, 1]
    tf.keras.layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
    
    # Convolutional layers
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    
    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    
    # Flatten and fully connected layers
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    
    # Final layer: number of neurons = number of classes
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# -------------------------------
# 6) Compile and Train
# -------------------------------
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Fix: Build the model explicitly so that weights are created
model.build(input_shape=(None, img_height, img_width, 3))

model.summary()

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=epochs
)

# -------------------------------
# 7) Save the trained model
# -------------------------------
model_save_path = "face_recognition_model.h5"
model.save(model_save_path)

print(f"Model saved to {model_save_path}")


Found 30 files belonging to 4 classes.
Using 24 files for training.
Detected classes: 4
Class names: ['aziz', 'mohammed', 'rashed', 'sultan']
Found 30 files belonging to 4 classes.
Using 6 files for validation.
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 64, 64, 3)         0         
                                                                 
 rescaling (Rescaling)       (None, 64, 64, 3)         0         
                                                                 
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2  (None, 31, 31, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 2

C:\Users\PCD\anaconda3\envs\tensorflow_env\lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [1]:
import cv2
import numpy as np
import tflite_runtime.interpreter as tflite

# Load class names
with open("class_names.txt", "r") as f:
    class_names = [line.strip() for line in f.readlines()]

# Load TFLite model
interpreter = tflite.Interpreter(model_path="face_recognition_model.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()
input_shape = input_details[0]['shape']

# Load Haar cascades
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
smile_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_smile.xml')

# Start video capture
cap = cv2.VideoCapture(0, cv2.CAP_V4L2)

if not cap.isOpened():
    print("❌ Webcam not detected.")
    exit()

confidence_threshold = 0.8

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Resize frame for performance (optional)
    frame = cv2.resize(frame, (320, 240))

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)

    for (x, y, w, h) in faces:
        face_roi = frame[y:y+h, x:x+w]
        face_gray = gray[y:y+h, x:x+w]

        # Preprocess for model input
        face_img = cv2.resize(face_roi, (64, 64))  # same as model input size
        face_img = face_img.astype(np.float32) / 255.0
        input_data = np.expand_dims(face_img, axis=0)

        # Set tensor and run inference
        interpreter.set_tensor(input_details[0]['index'], input_data)
        interpreter.invoke()
        prediction = interpreter.get_tensor(output_details[0]['index'])

        class_id = np.argmax(prediction)
        confidence = np.max(prediction)

        if confidence >= confidence_threshold:
            label = class_names[class_id]
            color = (0, 255, 0)
        else:
            label = "Unknown"
            color = (0, 0, 255)

        # Smile detection
        smiles = smile_cascade.detectMultiScale(face_gray, scaleFactor=1.8, minNeighbors=20)
        smile_label = "Smiling" if len(smiles) > 0 else "Not Smiling"

        # Draw results
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        cv2.putText(frame, f"{label}: {confidence:.2f}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        cv2.putText(frame, smile_label, (x, y + h + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)

    cv2.imshow("Face Recognition (TFLite)", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


❌ Error: No webcam detected or cannot be accessed.


In [1]:
import tensorflow as tf

# Load the .h5 model
model = tf.keras.models.load_model("face_recognition_model.h5")

# Convert to TFLite format
converter = tf.lite.TFLiteConverter.from_keras_model(model)
converter.optimizations = [tf.lite.Optimize.DEFAULT]  # Optional for size/speed
tflite_model = converter.convert()

# Save the .tflite model
with open("face_recognition_model.tflite", "wb") as f:
    f.write(tflite_model)

print("✅ Conversion complete.")


INFO:tensorflow:Assets written to: C:\Users\PCD\AppData\Local\Temp\tmp1gt3iplw\assets


INFO:tensorflow:Assets written to: C:\Users\PCD\AppData\Local\Temp\tmp1gt3iplw\assets


✅ Conversion complete.
